In [11]:
from time import sleep
import requests
from bs4 import BeautifulSoup
import json
import os
import glob
import csv
import pprint

In [5]:
areas = [
    "tokyo","osaka",
]

In [6]:
def make_new_dir():
    #ディレクトリ作成(最初)
    #data- jobs - エリアの数だけフォルダ
    #    - results - エリアの数だけフォルダ 
    
    for area in areas:
        new_dir_path_url = 'data/jobs/{}/'.format(area)
        new_dir_path_result = 'data/results/{}/'.format(area)
        os.makedirs(new_dir_path_url, exist_ok=True)
        os.makedirs(new_dir_path_result, exist_ok=True)

In [7]:
# 企業ごとの情報を取得
def __get_corp_info(job_id):
    sleep(1)
    url = '{}'
    url = url.format(job_id)

    try:
        r = requests.get(url)
        r.raise_for_status()
    except Exception as e:
        return None

    soup = BeautifulSoup(r.content, 'lxml')
    
    
    #店名取得
    try:
        store_name = soup.find(class_= 'rstinfo-table__name-wrap').get_text().strip()
    except Exception as e:
        store_name = None
        
    #ジャンル取得
    try:
        store_genure = soup.find("th",string="ジャンル").find_next_sibling("td").get_text().strip()
    except Exception as e:
        store_genure = None
    
    #住所取得
    try: 
        store_adress = soup.find("th",string="住所").find_next_sibling("td").find("p").get_text()
    except Exception as e:
        store_adress = None
      
    #電話番号取得
    try:
        store_phone_num = soup.find(class_ = 'rstdtl-booking-tel-modal__tel-num').get_text().strip()
    except Exception as e:
        store_phone_num = None
        
    #情報をまとめる 
    dict = {
            "name":store_name,
            "genure":store_genure,
            "address":store_adress,
            "phone":store_phone_num,
            }
    
    print(dict)
    return dict

In [13]:
# job_idを取得
def get_and_save_job_ids():    
    
    for area in areas:
        target_url = 'https://tabelog.com/{}/rstLst/1/'
        target_url = target_url.format(area)
        try:
            r = requests.get(target_url)
            r.raise_for_status()
        except Exception as e:
            return None

        soup = BeautifulSoup(r.content,'lxml')
        kensuu_number = int(soup.find(class_='c-page-count__line').find_next_sibling("span").get_text())
        print(kensuu_number)
        

        for i in range(1):  #(kensuu_number//20)+1
            page = str(i+1)
            print(page)
            url = 'https://tabelog.com/{}/rstLst/{}/'
            url = url.format(area,page)
            try:
                r = requests.get(url)
                r.raise_for_status()
            except Exception as e:
                return None

            job_ids = []
            soup = BeautifulSoup(r.content,'lxml')
            datas = soup.find_all(class_='list-rst__rst-name-target cpy-rst-name')
            
            for data in datas:
                job_id = data.get('href')
                print(job_id)
                job_ids.append(job_id)
            
            filepath = "data/jobs/{}/{}.csv".format(area, page)
            
            with open(filepath, mode="w") as f:
                writer = csv.writer(f)
                writer.writerow(job_ids)
                #f.write(json.dumps(job_ids))
                sleep(1)

In [9]:
# 企業の情報を一括で取得
def get_and_save_company_infos():
    for area in areas:
        filepaths = glob.glob("data/jobs/{}/*.json".format(area))
        all_job_ids = []


        for filepath in filepaths:
            with open(filepath) as f:
                job_ids = json.loads(f.read())
                print(job_ids)
            all_job_ids += job_ids


        for job_id in all_job_ids:
            file_id = area + job_id.split('/')[-2]
            out_filepath = "data/results/{}/{}.json".format(area,file_id)
            # if os.path.exists(out_filepath):
            #    print("job {} exists".format(file_id))
            #    continue

            corp_info = __get_corp_info(job_id)

            with open(out_filepath, "w") as f:
                f.write(json.dumps(corp_info,ensure_ascii=False,indent=2))

In [15]:
if __name__ == "__main__":
    make_new_dir()
    get_and_save_job_ids()
    #get_and_save_company_infos()

132371
1
https://tabelog.com/tokyo/A1304/A130404/13247945/
https://tabelog.com/tokyo/A1301/A130103/13110853/
https://tabelog.com/tokyo/A1325/A132502/13199567/
https://tabelog.com/tokyo/A1310/A131002/13269418/
https://tabelog.com/tokyo/A1314/A131402/13276345/
https://tabelog.com/tokyo/A1311/A131101/13094582/
https://tabelog.com/tokyo/A1305/A130501/13225658/
https://tabelog.com/tokyo/A1308/A130803/13196024/
https://tabelog.com/tokyo/A1305/A130501/13093141/
https://tabelog.com/tokyo/A1303/A130302/13139621/
https://tabelog.com/tokyo/A1301/A130103/13252068/
https://tabelog.com/tokyo/A1314/A131401/13191941/
https://tabelog.com/tokyo/A1301/A130103/13241455/
https://tabelog.com/tokyo/A1306/A130603/13279993/
https://tabelog.com/tokyo/A1314/A131403/13273070/
https://tabelog.com/tokyo/A1301/A130101/13213162/
https://tabelog.com/tokyo/A1324/A132402/13196646/
https://tabelog.com/tokyo/A1314/A131401/13115390/
https://tabelog.com/tokyo/A1315/A131501/13190507/
https://tabelog.com/tokyo/A1312/A131201/1